In [ ]:
import requests
import pandas as pd
from io import StringIO
import os

# 去除openssl的警告
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


def get_us_aid():
    print("Get data form web...")

    # API URL
    api_url = 'https://data.usaid.gov/resource/azij-hu6e.csv'

    # Get data from API
    response = requests.get(api_url)

    # Check if the request was successful
    if response.status_code == 200:
        csv_data = StringIO(response.text)
        with open("usaid.csv", "w") as f:
            f.write(csv_data.getvalue())
    else:
        print(f'Failed to get data. Status code: {response.status_code}')


if not os.path.exists("usaid.csv"):
    get_us_aid()

In [ ]:
df = pd.read_csv("usaid.csv")
# 将df.columns转换为list
columns_list = df.columns.tolist()
# 将list_name按照首字母排序, inplace=True表示直接在原list上进行排序
columns_list.sort(key=lambda x: x[0])
# 将list转换为string
columns_string = "\n".join(columns_list)

# 使用SODA api获取数据
# https://dev.socrata.com/foundry/data.usaid.gov/azij-hu6e
# 构建目标URL(包含country_name, foreign_assistance_objective_1)
api_url = "https://data.usaid.gov/resource/azij-hu6e.json?$select=country_name,foreign_assistance_objective_1"
response = requests.get(api_url)
if response.status_code == 200:
    json_data = response.json()
    # 将json_data转换为df
    df = pd.DataFrame(json_data)
    print(df.drop_duplicates())
else:
    print(f'Failed to get data. Status code: {response.status_code}')

In [ ]:
def get_account_by_country(country_name, all_data):
    single = all_data[all_data["Country Name"] == country_name]
    return single

In [ ]:
import matplotlib.pyplot as plt
import streamlit as st

df = pd.read_csv("./data/aid_summary.csv")
# 提取阿富汗的所有援助拨款数据
# afghanistan = df[df["Country Name"] == "Afghanistan"]
afghanistan = get_account_by_country("Afghanistan", df)
# 展示数据, 按照年份从左至右, 柱形图每个年份应该包含两类(Economic and Military)
# 将同一年的柱形图放在一起, 以便比较, 颜色也区分开
plt.figure(figsize=(12, 6))

bar_width = 0.35  # 条形宽度

# 绘制 Economic 的条形
economic_data = afghanistan[afghanistan['Foreign Assistance Objective Name'] == 'Economic']
plt.bar(economic_data['Fiscal Year'] - bar_width / 2, economic_data['Total Aid Disbursement'], bar_width, label='Economic', color='blue')

# 绘制 Military 的条形
military_data = afghanistan[afghanistan['Foreign Assistance Objective Name'] == 'Military']
plt.bar(military_data['Fiscal Year'] + bar_width / 2, military_data['Total Aid Disbursement'], bar_width, label='Military', color='green')

plt.xlabel('Fiscal Year')
plt.ylabel('Total Aid Disbursement')
plt.title('Aid to Afghanistan by Year and Objective (with missing Military aid years)')
plt.xticks(afghanistan['Fiscal Year'].unique())
plt.legend(title='Foreign Assistance Objective Name')
plt.tight_layout()

# 在 Streamlit 应用中显示图表
st.pyplot(plt)

In [1]:
import Data

target_dataset = "k87i-9i5x"
df = Data.get_all_data(target_dataset)

/Users/luyao/PycharmProjects/ForeignAssistance/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


KeyError: ('Country Name', 'Fiscal Year', 'Transaction Type Name', 'current_amount')

In [13]:
from sympy import Matrix
'''
\begin{array}{r}
-x_2+x_3+x_4=2 \\
x_1+2 x_2+2 x_3-x_4=3 \\
x_1+3 x_2+x_3=2
\end{array}
'''
# 转换为矩阵(方程组的增广矩阵, 有的系数是零, 你需要按照顺序填写)
m = Matrix([[0, -1, 1, 1, 2], [1, 2, 2, -1, 3], [1, 3, 1, 0, 2]])
# 求解
m.rref()

(Matrix([
 [1, 0,  4, 0, 13/2],
 [0, 1, -1, 0, -3/2],
 [0, 0,  0, 1,  1/2]]),
 (0, 1, 3))

In [1]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.usaid.gov", "7wQTG5bIYVjjEUv5JZ3KLJvEg")

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.usaid.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get_all("azij-hu6e")

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

results_df.to_csv("usaid.csv", index=False)

/Users/luyao/PycharmProjects/ForeignAssistance/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


KeyboardInterrupt: 